# Cancer Type Classification from Gene Expression Data

## Project Overview
This project builds a machine learning pipeline to classify cancer types based on RNA-Seq gene expression patterns from the TCGA PANCAN dataset.

**Dataset:** TCGA PANCAN HiSeq (801 samples x 20,531 genes)

**Cancer Types:**
- BRCA: Breast Invasive Carcinoma
- COAD: Colon Adenocarcinoma
- KIRC: Kidney Renal Clear Cell Carcinoma
- LUAD: Lung Adenocarcinoma
- PRAD: Prostate Adenocarcinoma

---

## Table of Contents
1. [Setup and Imports](#1.-Setup-and-Imports)
2. [Step 1: Load and Explore Data (EDA)](#2.-Step-1:-Load-and-Explore-Data-(EDA))
3. [Step 2: Data Preprocessing](#3.-Step-2:-Data-Preprocessing)
4. [Step 3: Dimensionality Reduction (PCA)](#4.-Step-3:-Dimensionality-Reduction-(PCA))
5. [Step 4: Model Building](#5.-Step-4:-Model-Building)
6. [Step 5: Model Evaluation](#6.-Step-5:-Model-Evaluation)
7. [Step 6: Feature Importance Analysis](#7.-Step-6:-Feature-Importance-Analysis)
8. [Step 7: Executive Summary](#8.-Step-7:-Executive-Summary)

---
## 1. Setup and Imports

In [ ]:
# Install required packages if not already installed
# !pip install xgboost

In [ ]:
# =============================================================================
# IMPORT LIBRARIES
# =============================================================================

# Data manipulation and numerical operations
import numpy as np
import pandas as pd

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Preprocessing
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA

# Classification Models
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier

# Evaluation Metrics
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, roc_curve, auc,
    confusion_matrix, classification_report,
    precision_recall_curve, average_precision_score
)

# Suppress warnings for cleaner output
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

# Set display options
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 200)

# Set plot style - use a compatible style
try:
    plt.style.use('seaborn-v0_8-whitegrid')
except:
    try:
        plt.style.use('seaborn-whitegrid')
    except:
        plt.style.use('ggplot')

sns.set_palette('husl')

# Compatibility: define display function for non-Jupyter environments
try:
    from IPython.display import display
except ImportError:
    def display(x):
        print(x)

print("All libraries imported successfully!")

---
## 2. Step 1: Load and Explore Data (EDA)

In this section, we load the TCGA PANCAN dataset and perform exploratory data analysis to understand the structure, distribution, and characteristics of the data.

In [ ]:
# =============================================================================
# LOAD DATA
# =============================================================================

# Define file paths
data_path = "TCGA-PANCAN-HiSeq-801x20531/TCGA-PANCAN-HiSeq-801x20531/data.csv"
labels_path = "TCGA-PANCAN-HiSeq-801x20531/TCGA-PANCAN-HiSeq-801x20531/labels.csv"

# Load gene expression data and labels
print("Loading data...")
data = pd.read_csv(data_path, index_col=0)
labels = pd.read_csv(labels_path, index_col=0)

print(f"\nData loaded successfully!")
print(f"Gene Expression Data Shape: {data.shape}")
print(f"Labels Shape: {labels.shape}")

In [ ]:
# =============================================================================
# DATASET OVERVIEW
# =============================================================================

print("=" * 70)
print("DATASET OVERVIEW")
print("=" * 70)

print(f"\nNumber of Samples: {data.shape[0]}")
print(f"Number of Features (Genes): {data.shape[1]}")
print(f"Number of Cancer Types: {labels['Class'].nunique()}")

print("\n--- First 5 Samples ---")
display(data.iloc[:5, :10])  # Show first 10 genes for first 5 samples

print("\n--- Labels Preview ---")
display(labels.head(10))

In [ ]:
# =============================================================================
# CLASS DISTRIBUTION ANALYSIS
# =============================================================================

print("=" * 70)
print("CLASS DISTRIBUTION")
print("=" * 70)

# Calculate class distribution
class_distribution = labels['Class'].value_counts()
class_percentages = labels['Class'].value_counts(normalize=True) * 100

print("\nCancer Type Distribution:")
print("-" * 40)
for cancer_type in class_distribution.index:
    count = class_distribution[cancer_type]
    pct = class_percentages[cancer_type]
    print(f"{cancer_type}: {count:4d} samples ({pct:5.1f}%)")

# Visualize class distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Bar chart
colors = sns.color_palette('husl', len(class_distribution))
bars = axes[0].bar(class_distribution.index, class_distribution.values, color=colors, edgecolor='black')
axes[0].set_xlabel('Cancer Type', fontsize=12)
axes[0].set_ylabel('Number of Samples', fontsize=12)
axes[0].set_title('Distribution of Cancer Types', fontsize=14, fontweight='bold')
for bar, count in zip(bars, class_distribution.values):
    axes[0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 5,
                 str(count), ha='center', va='bottom', fontsize=11, fontweight='bold')

# Pie chart
wedges, texts, autotexts = axes[1].pie(class_distribution.values, labels=class_distribution.index,
                                        autopct='%1.1f%%', colors=colors, explode=[0.02]*5,
                                        shadow=True, startangle=90)
axes[1].set_title('Cancer Type Proportions', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.savefig('01_class_distribution.png', dpi=150, bbox_inches='tight')
plt.show()

# Check for class imbalance
imbalance_ratio = class_distribution.max() / class_distribution.min()
print(f"\nClass Imbalance Ratio (max/min): {imbalance_ratio:.2f}")
if imbalance_ratio > 3:
    print("Note: Moderate class imbalance detected. Will use stratified sampling.")

In [ ]:
# =============================================================================
# DATA QUALITY CHECK
# =============================================================================

print("=" * 70)
print("DATA QUALITY CHECK")
print("=" * 70)

# Check for missing values
missing_values = data.isnull().sum().sum()
missing_percentage = (missing_values / data.size) * 100
print(f"\nMissing Values: {missing_values} ({missing_percentage:.4f}%)")

# Check for duplicate samples
duplicate_samples = data.duplicated().sum()
print(f"Duplicate Samples: {duplicate_samples}")

# Check data types
print(f"\nData Types:")
print(data.dtypes.value_counts())

# Check for constant features (zero variance)
constant_features = (data.std() == 0).sum()
print(f"\nConstant Features (zero variance): {constant_features}")

In [ ]:
# =============================================================================
# STATISTICAL SUMMARY
# =============================================================================

print("=" * 70)
print("STATISTICAL SUMMARY")
print("=" * 70)

# Basic statistics for gene expression values
print("\n--- Gene Expression Statistics (First 10 Genes) ---")
display(data.iloc[:, :10].describe())

# Overall statistics
print("\n--- Overall Gene Expression Distribution ---")
all_values = data.values.flatten()
print(f"Min: {all_values.min():.4f}")
print(f"Max: {all_values.max():.4f}")
print(f"Mean: {all_values.mean():.4f}")
print(f"Median: {np.median(all_values):.4f}")
print(f"Std Dev: {all_values.std():.4f}")

In [ ]:
# =============================================================================
# GENE EXPRESSION DISTRIBUTION VISUALIZATION
# =============================================================================

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Distribution of mean expression per gene
gene_means = data.mean(axis=0)
axes[0, 0].hist(gene_means, bins=50, color='steelblue', edgecolor='black', alpha=0.7)
axes[0, 0].set_xlabel('Mean Expression', fontsize=12)
axes[0, 0].set_ylabel('Number of Genes', fontsize=12)
axes[0, 0].set_title('Distribution of Mean Expression Across Genes', fontsize=14, fontweight='bold')
axes[0, 0].axvline(gene_means.mean(), color='red', linestyle='--', label=f'Mean: {gene_means.mean():.2f}')
axes[0, 0].legend()

# Distribution of variance per gene
gene_vars = data.var(axis=0)
axes[0, 1].hist(gene_vars[gene_vars > 0], bins=50, color='coral', edgecolor='black', alpha=0.7)
axes[0, 1].set_xlabel('Variance', fontsize=12)
axes[0, 1].set_ylabel('Number of Genes', fontsize=12)
axes[0, 1].set_title('Distribution of Variance Across Genes', fontsize=14, fontweight='bold')

# Box plot of expression by cancer type (sample of genes)
# Select highly variable genes
top_var_genes = gene_vars.nlargest(5).index.tolist()
sample_data = data[top_var_genes].copy()
sample_data['Cancer_Type'] = labels['Class'].values
sample_melted = sample_data.melt(id_vars='Cancer_Type', var_name='Gene', value_name='Expression')

sns.boxplot(data=sample_melted, x='Cancer_Type', y='Expression', hue='Gene', ax=axes[1, 0])
axes[1, 0].set_xlabel('Cancer Type', fontsize=12)
axes[1, 0].set_ylabel('Expression Level', fontsize=12)
axes[1, 0].set_title('Expression of Top 5 Variable Genes by Cancer Type', fontsize=14, fontweight='bold')
axes[1, 0].legend(title='Gene', bbox_to_anchor=(1.02, 1), loc='upper left')

# Correlation heatmap of top variable genes
top_20_genes = gene_vars.nlargest(20).index.tolist()
corr_matrix = data[top_20_genes].corr()
sns.heatmap(corr_matrix, cmap='RdBu_r', center=0, ax=axes[1, 1],
            xticklabels=False, yticklabels=False, cbar_kws={'shrink': 0.8})
axes[1, 1].set_title('Correlation Heatmap (Top 20 Variable Genes)', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.savefig('02_eda_visualizations.png', dpi=150, bbox_inches='tight')
plt.show()

---
## 3. Step 2: Data Preprocessing

In this section, we prepare the data for modeling:
- Handle missing values (if any)
- Encode target labels
- Normalize features using StandardScaler
- Split into training and test sets

In [ ]:
# =============================================================================
# HANDLE MISSING VALUES
# =============================================================================

print("=" * 70)
print("DATA PREPROCESSING")
print("=" * 70)

# Check and handle missing values
if data.isnull().sum().sum() > 0:
    print("\nHandling missing values with median imputation...")
    data = data.fillna(data.median())
    print("Missing values filled.")
else:
    print("\nNo missing values found.")

In [ ]:
# =============================================================================
# PREPARE FEATURES AND TARGET
# =============================================================================

# Extract features (X) and target (y)
X = data.values
y = labels['Class'].values

print(f"\nFeatures shape: {X.shape}")
print(f"Target shape: {y.shape}")

# Encode target labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Store class names for later use
class_names = label_encoder.classes_
n_classes = len(class_names)

print(f"\nClass Encoding:")
for i, name in enumerate(class_names):
    print(f"  {name}: {i}")

In [ ]:
# =============================================================================
# FEATURE NORMALIZATION
# =============================================================================

print("\nNormalizing features with StandardScaler...")

# Apply StandardScaler to normalize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print(f"\nBefore Scaling:")
print(f"  Mean: {X.mean():.4f}")
print(f"  Std:  {X.std():.4f}")

print(f"\nAfter Scaling:")
print(f"  Mean: {X_scaled.mean():.6f}")
print(f"  Std:  {X_scaled.std():.4f}")

---
## 4. Step 3: Dimensionality Reduction (PCA)

With 20,531 features, dimensionality reduction is essential to:
- Reduce computational complexity
- Mitigate the curse of dimensionality
- Remove noise and redundant features
- Improve model generalization

In [ ]:
# =============================================================================
# PCA ANALYSIS - VARIANCE EXPLAINED
# =============================================================================

print("=" * 70)
print("DIMENSIONALITY REDUCTION (PCA)")
print("=" * 70)

# Fit PCA to understand variance distribution
pca_full = PCA(random_state=RANDOM_STATE)
pca_full.fit(X_scaled)

# Calculate cumulative explained variance
cumulative_variance = np.cumsum(pca_full.explained_variance_ratio_)

# Find optimal number of components for different thresholds
print("\nComponents needed for variance retention:")
print("-" * 40)
thresholds = [0.80, 0.85, 0.90, 0.95, 0.99]
for threshold in thresholds:
    n_comp = np.argmax(cumulative_variance >= threshold) + 1
    print(f"  {threshold*100:.0f}% variance: {n_comp} components")

In [ ]:
# =============================================================================
# VISUALIZE PCA VARIANCE
# =============================================================================

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Individual variance explained
n_show = 50
axes[0].bar(range(1, n_show + 1), pca_full.explained_variance_ratio_[:n_show] * 100,
            color='steelblue', edgecolor='black', alpha=0.8)
axes[0].set_xlabel('Principal Component', fontsize=12)
axes[0].set_ylabel('Explained Variance (%)', fontsize=12)
axes[0].set_title('Variance Explained by Each PC (First 50)', fontsize=14, fontweight='bold')

# Cumulative variance explained
n_cumulative = 300
axes[1].plot(range(1, n_cumulative + 1), cumulative_variance[:n_cumulative] * 100,
             'b-', linewidth=2, label='Cumulative Variance')
axes[1].axhline(y=95, color='red', linestyle='--', linewidth=1.5, label='95% Threshold')
axes[1].axhline(y=90, color='orange', linestyle='--', linewidth=1.5, label='90% Threshold')
axes[1].set_xlabel('Number of Components', fontsize=12)
axes[1].set_ylabel('Cumulative Explained Variance (%)', fontsize=12)
axes[1].set_title('Cumulative Explained Variance', fontsize=14, fontweight='bold')
axes[1].legend(loc='lower right')
axes[1].set_ylim(0, 105)

plt.tight_layout()
plt.savefig('03_pca_variance.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# =============================================================================
# APPLY PCA TRANSFORMATION
# =============================================================================

# Choose number of components (95% variance retained)
n_components = np.argmax(cumulative_variance >= 0.95) + 1
print(f"\nApplying PCA with {n_components} components (95% variance retained)")

# Fit and transform
pca = PCA(n_components=n_components, random_state=RANDOM_STATE)
X_pca = pca.fit_transform(X_scaled)

print(f"\nOriginal shape: {X_scaled.shape}")
print(f"Reduced shape:  {X_pca.shape}")
print(f"Dimensionality reduction: {X_scaled.shape[1]} -> {X_pca.shape[1]} ({(1 - X_pca.shape[1]/X_scaled.shape[1])*100:.1f}% reduction)")

In [ ]:
# =============================================================================
# VISUALIZE PCA - 2D AND 3D PROJECTIONS
# =============================================================================

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# 2D PCA Plot
colors = sns.color_palette('husl', n_classes)
for i, (cancer_type, color) in enumerate(zip(class_names, colors)):
    mask = y_encoded == i
    axes[0].scatter(X_pca[mask, 0], X_pca[mask, 1], c=[color], label=cancer_type,
                    alpha=0.7, s=50, edgecolors='white', linewidths=0.5)

axes[0].set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]*100:.1f}% variance)', fontsize=12)
axes[0].set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]*100:.1f}% variance)', fontsize=12)
axes[0].set_title('Cancer Samples in 2D PCA Space', fontsize=14, fontweight='bold')
axes[0].legend(title='Cancer Type', loc='best')

# PC1 vs PC3
for i, (cancer_type, color) in enumerate(zip(class_names, colors)):
    mask = y_encoded == i
    axes[1].scatter(X_pca[mask, 0], X_pca[mask, 2], c=[color], label=cancer_type,
                    alpha=0.7, s=50, edgecolors='white', linewidths=0.5)

axes[1].set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]*100:.1f}% variance)', fontsize=12)
axes[1].set_ylabel(f'PC3 ({pca.explained_variance_ratio_[2]*100:.1f}% variance)', fontsize=12)
axes[1].set_title('Cancer Samples: PC1 vs PC3', fontsize=14, fontweight='bold')
axes[1].legend(title='Cancer Type', loc='best')

plt.tight_layout()
plt.savefig('04_pca_visualization.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# =============================================================================
# 3D PCA VISUALIZATION
# =============================================================================

from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(12, 10))
ax = fig.add_subplot(111, projection='3d')

for i, (cancer_type, color) in enumerate(zip(class_names, colors)):
    mask = y_encoded == i
    ax.scatter(X_pca[mask, 0], X_pca[mask, 1], X_pca[mask, 2],
               c=[color], label=cancer_type, alpha=0.7, s=50)

ax.set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]*100:.1f}%)')
ax.set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]*100:.1f}%)')
ax.set_zlabel(f'PC3 ({pca.explained_variance_ratio_[2]*100:.1f}%)')
ax.set_title('Cancer Samples in 3D PCA Space', fontsize=14, fontweight='bold')
ax.legend(title='Cancer Type', loc='upper left')

plt.tight_layout()
plt.savefig('05_pca_3d.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# =============================================================================
# TRAIN-TEST SPLIT
# =============================================================================

print("\n" + "=" * 70)
print("TRAIN-TEST SPLIT")
print("=" * 70)

# Split data with stratification to maintain class proportions
X_train, X_test, y_train, y_test = train_test_split(
    X_pca, y_encoded, 
    test_size=0.2, 
    random_state=RANDOM_STATE, 
    stratify=y_encoded
)

print(f"\nTraining Set: {X_train.shape[0]} samples ({X_train.shape[0]/len(y_encoded)*100:.0f}%)")
print(f"Test Set: {X_test.shape[0]} samples ({X_test.shape[0]/len(y_encoded)*100:.0f}%)")

# Verify stratification
print("\nClass distribution in splits:")
print("-" * 50)
for i, name in enumerate(class_names):
    train_count = (y_train == i).sum()
    test_count = (y_test == i).sum()
    print(f"{name}: Train={train_count}, Test={test_count}")

---
## 5. Step 4: Model Building

We will train and compare multiple classification models:
1. **Logistic Regression** (Baseline)
2. **Decision Tree Classifier**
3. **Random Forest Classifier**
4. **XGBoost Classifier**
5. **Gradient Boosting Classifier**

We will also perform hyperparameter tuning using GridSearchCV.

In [ ]:
# =============================================================================
# DEFINE MODELS
# =============================================================================

print("=" * 70)
print("MODEL BUILDING")
print("=" * 70)

# Define all models with their configurations
models = {
    'Logistic Regression': LogisticRegression(
        max_iter=1000,
        random_state=RANDOM_STATE,
        multi_class='multinomial',
        solver='lbfgs'
    ),
    'Decision Tree': DecisionTreeClassifier(
        max_depth=15,
        min_samples_split=5,
        random_state=RANDOM_STATE
    ),
    'Random Forest': RandomForestClassifier(
        n_estimators=200,
        max_depth=20,
        min_samples_split=5,
        random_state=RANDOM_STATE,
        n_jobs=1
    ),
    'XGBoost': XGBClassifier(
        n_estimators=200,
        max_depth=6,
        learning_rate=0.1,
        random_state=RANDOM_STATE,
        use_label_encoder=False,
        eval_metric='mlogloss',
        n_jobs=1
    ),
    'Gradient Boosting': GradientBoostingClassifier(
        n_estimators=200,
        max_depth=6,
        learning_rate=0.1,
        random_state=RANDOM_STATE
    )
}

print(f"\nModels to train: {len(models)}")
for name in models:
    print(f"  - {name}")

In [ ]:
# =============================================================================
# TRAIN ALL MODELS
# =============================================================================

# Cross-validation setup
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)

# Store results
results = {}

print("\nTraining models...\n")

for name, model in models.items():
    print(f"{'='*60}")
    print(f"Training: {name}")
    print(f"{'='*60}")
    
    # Cross-validation scores
    cv_scores = cross_val_score(model, X_train, y_train, cv=cv, scoring='accuracy', n_jobs=1)
    print(f"  CV Accuracy: {cv_scores.mean():.4f} (+/- {cv_scores.std()*2:.4f})")
    
    # Train on full training set
    model.fit(X_train, y_train)
    
    # Predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Probability predictions (for AUC-ROC)
    if hasattr(model, 'predict_proba'):
        y_test_proba = model.predict_proba(X_test)
    else:
        y_test_proba = None
    
    # Calculate metrics
    train_accuracy = accuracy_score(y_train, y_train_pred)
    test_accuracy = accuracy_score(y_test, y_test_pred)
    
    print(f"  Train Accuracy: {train_accuracy:.4f}")
    print(f"  Test Accuracy:  {test_accuracy:.4f}")
    
    # Store results
    results[name] = {
        'model': model,
        'cv_scores': cv_scores,
        'train_accuracy': train_accuracy,
        'test_accuracy': test_accuracy,
        'y_train_pred': y_train_pred,
        'y_test_pred': y_test_pred,
        'y_test_proba': y_test_proba
    }
    
    # Check for overfitting
    overfit_gap = train_accuracy - test_accuracy
    if overfit_gap > 0.1:
        print(f"  WARNING: Possible overfitting (gap: {overfit_gap:.4f})")
    
    print()

In [ ]:
# =============================================================================
# HYPERPARAMETER TUNING WITH GRIDSEARCHCV
# =============================================================================

print("\n" + "=" * 70)
print("HYPERPARAMETER TUNING (GridSearchCV)")
print("=" * 70)

# Tune XGBoost using GridSearchCV
print("\nTuning XGBoost hyperparameters...")

# Define parameter grid
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [4, 6, 8],
    'learning_rate': [0.05, 0.1, 0.2]
}

# Create base model
xgb_base = XGBClassifier(
    random_state=RANDOM_STATE,
    use_label_encoder=False,
    eval_metric='mlogloss',
    n_jobs=1
)

# GridSearchCV
grid_search = GridSearchCV(
    estimator=xgb_base,
    param_grid=param_grid,
    cv=3,
    scoring='accuracy',
    n_jobs=1,
    verbose=1
)

# Fit GridSearchCV
grid_search.fit(X_train, y_train)

print(f"\nBest Parameters: {grid_search.best_params_}")
print(f"Best CV Score: {grid_search.best_score_:.4f}")

# Update XGBoost with best parameters
best_xgb = grid_search.best_estimator_
y_test_pred_tuned = best_xgb.predict(X_test)
tuned_accuracy = accuracy_score(y_test, y_test_pred_tuned)

print(f"\nTuned XGBoost Test Accuracy: {tuned_accuracy:.4f}")
print(f"Original XGBoost Test Accuracy: {results['XGBoost']['test_accuracy']:.4f}")
print(f"Improvement: {(tuned_accuracy - results['XGBoost']['test_accuracy'])*100:.2f}%")

# Update results with tuned model
results['XGBoost (Tuned)'] = {
    'model': best_xgb,
    'cv_scores': np.array([grid_search.best_score_] * 5),
    'train_accuracy': accuracy_score(y_train, best_xgb.predict(X_train)),
    'test_accuracy': tuned_accuracy,
    'y_train_pred': best_xgb.predict(X_train),
    'y_test_pred': y_test_pred_tuned,
    'y_test_proba': best_xgb.predict_proba(X_test)
}

In [ ]:
# =============================================================================
# DUMMY SAMPLE PREDICTION
# =============================================================================

print("\n" + "=" * 70)
print("DUMMY SAMPLE PREDICTION EXAMPLE")
print("=" * 70)

# Create a dummy sample by taking the mean of each cancer type
print("\nCreating dummy sample from class centroids...\n")

# Get the best model (Logistic Regression performed best)
best_model_name = max(results, key=lambda x: results[x]['test_accuracy'])
best_model = results[best_model_name]['model']

# Create dummy samples (one per class centroid)
for i, cancer_type in enumerate(class_names):
    # Get class centroid in PCA space
    class_mask = y_encoded == i
    centroid = X_pca[class_mask].mean(axis=0).reshape(1, -1)
    
    # Predict
    prediction = best_model.predict(centroid)
    predicted_class = class_names[prediction[0]]
    
    # Get probability
    if hasattr(best_model, 'predict_proba'):
        proba = best_model.predict_proba(centroid)[0]
        confidence = proba[prediction[0]] * 100
    else:
        confidence = 'N/A'
    
    print(f"Dummy sample (centroid of {cancer_type}):")
    print(f"  -> Predicted: {predicted_class}")
    print(f"  -> Confidence: {confidence:.1f}%")
    print(f"  -> Correct: {'Yes' if predicted_class == cancer_type else 'No'}")
    print()

# Also predict on a random test sample
print("\n--- Random Test Sample Prediction ---")
random_idx = np.random.randint(0, len(X_test))
sample = X_test[random_idx].reshape(1, -1)
true_label = class_names[y_test[random_idx]]
pred_label = class_names[best_model.predict(sample)[0]]

print(f"Sample Index: {random_idx}")
print(f"True Label: {true_label}")
print(f"Predicted Label: {pred_label}")

if hasattr(best_model, 'predict_proba'):
    proba = best_model.predict_proba(sample)[0]
    print(f"\nPrediction Probabilities:")
    for name, prob in zip(class_names, proba):
        print(f"  {name}: {prob*100:.2f}%")

---
## 6. Step 5: Model Evaluation

Comprehensive evaluation using multiple metrics:
- Accuracy
- Precision, Recall, F1-Score
- AUC-ROC
- TPR (True Positive Rate) / TNR (True Negative Rate)
- Confusion Matrix

In [ ]:
# =============================================================================
# COMPREHENSIVE MODEL EVALUATION
# =============================================================================

print("=" * 70)
print("MODEL EVALUATION")
print("=" * 70)

# Function to calculate all metrics
def calculate_metrics(y_true, y_pred, y_proba=None, class_names=None):
    """Calculate comprehensive classification metrics."""
    metrics = {}
    
    # Basic metrics
    metrics['Accuracy'] = accuracy_score(y_true, y_pred)
    metrics['Precision (Weighted)'] = precision_score(y_true, y_pred, average='weighted')
    metrics['Recall (Weighted)'] = recall_score(y_true, y_pred, average='weighted')
    metrics['F1 (Weighted)'] = f1_score(y_true, y_pred, average='weighted')
    metrics['Precision (Macro)'] = precision_score(y_true, y_pred, average='macro')
    metrics['Recall (Macro)'] = recall_score(y_true, y_pred, average='macro')
    metrics['F1 (Macro)'] = f1_score(y_true, y_pred, average='macro')
    
    # AUC-ROC (One-vs-Rest)
    if y_proba is not None:
        try:
            metrics['AUC-ROC (OvR)'] = roc_auc_score(y_true, y_proba, multi_class='ovr')
        except:
            metrics['AUC-ROC (OvR)'] = np.nan
    else:
        metrics['AUC-ROC (OvR)'] = np.nan
    
    # Calculate TPR and TNR for each class
    cm = confusion_matrix(y_true, y_pred)
    n_classes = len(cm)
    
    tpr_list = []  # Sensitivity / Recall
    tnr_list = []  # Specificity
    
    for i in range(n_classes):
        tp = cm[i, i]
        fn = cm[i, :].sum() - tp
        fp = cm[:, i].sum() - tp
        tn = cm.sum() - tp - fn - fp
        
        tpr = tp / (tp + fn) if (tp + fn) > 0 else 0
        tnr = tn / (tn + fp) if (tn + fp) > 0 else 0
        
        tpr_list.append(tpr)
        tnr_list.append(tnr)
    
    metrics['TPR (Avg)'] = np.mean(tpr_list)
    metrics['TNR (Avg)'] = np.mean(tnr_list)
    
    return metrics

# Calculate metrics for all models
evaluation_results = []

for name, result in results.items():
    metrics = calculate_metrics(
        y_test, 
        result['y_test_pred'], 
        result['y_test_proba'],
        class_names
    )
    metrics['Model'] = name
    evaluation_results.append(metrics)

# Create DataFrame
eval_df = pd.DataFrame(evaluation_results)
eval_df = eval_df.set_index('Model')

# Display results
print("\n--- Comprehensive Model Comparison ---\n")
display(eval_df.round(4))

In [ ]:
# =============================================================================
# VISUALIZE MODEL COMPARISON
# =============================================================================

# Select key metrics for visualization
key_metrics = ['Accuracy', 'F1 (Weighted)', 'AUC-ROC (OvR)', 'TPR (Avg)', 'TNR (Avg)']

fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.flatten()

model_names = eval_df.index.tolist()
x = np.arange(len(model_names))
colors = sns.color_palette('husl', len(model_names))

for idx, metric in enumerate(key_metrics):
    values = eval_df[metric].values
    bars = axes[idx].bar(x, values, color=colors, edgecolor='black')
    axes[idx].set_xlabel('Model', fontsize=11)
    axes[idx].set_ylabel(metric, fontsize=11)
    axes[idx].set_title(f'{metric} by Model', fontsize=12, fontweight='bold')
    axes[idx].set_xticks(x)
    axes[idx].set_xticklabels(model_names, rotation=45, ha='right', fontsize=9)
    axes[idx].set_ylim(0, 1.05)
    
    # Add value labels
    for bar, val in zip(bars, values):
        if not np.isnan(val):
            axes[idx].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                          f'{val:.3f}', ha='center', va='bottom', fontsize=8)

# Summary comparison in last subplot
axes[5].axis('off')
best_model_idx = eval_df['F1 (Weighted)'].idxmax()
summary_text = f"""BEST MODEL: {best_model_idx}

Key Metrics:
- Accuracy: {eval_df.loc[best_model_idx, 'Accuracy']:.4f}
- F1 Score: {eval_df.loc[best_model_idx, 'F1 (Weighted)']:.4f}
- AUC-ROC: {eval_df.loc[best_model_idx, 'AUC-ROC (OvR)']:.4f}
- TPR: {eval_df.loc[best_model_idx, 'TPR (Avg)']:.4f}
- TNR: {eval_df.loc[best_model_idx, 'TNR (Avg)']:.4f}
"""
axes[5].text(0.1, 0.5, summary_text, transform=axes[5].transAxes,
             fontsize=12, verticalalignment='center',
             bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.5))

plt.tight_layout()
plt.savefig('06_model_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# =============================================================================
# CONFUSION MATRICES
# =============================================================================

# Plot confusion matrices for top 4 models
top_models = ['Logistic Regression', 'Random Forest', 'XGBoost', 'Gradient Boosting']

fig, axes = plt.subplots(2, 2, figsize=(14, 12))
axes = axes.flatten()

for idx, model_name in enumerate(top_models):
    y_pred = results[model_name]['y_test_pred']
    cm = confusion_matrix(y_test, y_pred)
    
    # Calculate percentages
    cm_pct = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis] * 100
    
    # Create annotations with both count and percentage
    annotations = np.empty_like(cm, dtype=object)
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            annotations[i, j] = f'{cm[i, j]}\n({cm_pct[i, j]:.1f}%)'
    
    sns.heatmap(cm, annot=annotations, fmt='', cmap='Blues', ax=axes[idx],
                xticklabels=class_names, yticklabels=class_names,
                cbar_kws={'shrink': 0.8})
    axes[idx].set_xlabel('Predicted', fontsize=11)
    axes[idx].set_ylabel('Actual', fontsize=11)
    acc = results[model_name]['test_accuracy']
    axes[idx].set_title(f'{model_name}\nAccuracy: {acc:.4f}', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.savefig('07_confusion_matrices.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# =============================================================================
# ROC CURVES (One-vs-Rest)
# =============================================================================

from sklearn.preprocessing import label_binarize

# Binarize the output for ROC curve
y_test_bin = label_binarize(y_test, classes=range(n_classes))

fig, axes = plt.subplots(2, 2, figsize=(14, 12))
axes = axes.flatten()

for idx, model_name in enumerate(top_models):
    y_proba = results[model_name]['y_test_proba']
    
    if y_proba is not None:
        # Plot ROC curve for each class
        for i, (class_name, color) in enumerate(zip(class_names, colors)):
            fpr, tpr, _ = roc_curve(y_test_bin[:, i], y_proba[:, i])
            roc_auc = auc(fpr, tpr)
            axes[idx].plot(fpr, tpr, color=color, lw=2,
                          label=f'{class_name} (AUC = {roc_auc:.3f})')
        
        # Plot diagonal
        axes[idx].plot([0, 1], [0, 1], 'k--', lw=1.5, label='Random')
        axes[idx].set_xlim([0.0, 1.0])
        axes[idx].set_ylim([0.0, 1.05])
        axes[idx].set_xlabel('False Positive Rate', fontsize=11)
        axes[idx].set_ylabel('True Positive Rate', fontsize=11)
        axes[idx].set_title(f'ROC Curves - {model_name}', fontsize=12, fontweight='bold')
        axes[idx].legend(loc='lower right', fontsize=9)

plt.tight_layout()
plt.savefig('08_roc_curves.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# =============================================================================
# DETAILED CLASSIFICATION REPORT FOR BEST MODEL
# =============================================================================

best_model_name = eval_df['F1 (Weighted)'].idxmax()
print(f"\n{'='*70}")
print(f"DETAILED CLASSIFICATION REPORT - {best_model_name}")
print(f"{'='*70}\n")

print(classification_report(y_test, results[best_model_name]['y_test_pred'], 
                           target_names=class_names, digits=4))

In [ ]:
# =============================================================================
# OVERFITTING/UNDERFITTING ANALYSIS
# =============================================================================

print("\n" + "=" * 70)
print("OVERFITTING/UNDERFITTING ANALYSIS")
print("=" * 70)

print("\n{:<25} {:>15} {:>15} {:>15} {:>15}".format(
    'Model', 'Train Acc', 'Test Acc', 'Gap', 'Status'))
print("-" * 85)

for name, result in results.items():
    train_acc = result['train_accuracy']
    test_acc = result['test_accuracy']
    gap = train_acc - test_acc
    
    if gap > 0.15:
        status = 'OVERFITTING'
    elif test_acc < 0.85:
        status = 'UNDERFITTING'
    else:
        status = 'Good Fit'
    
    print("{:<25} {:>15.4f} {:>15.4f} {:>15.4f} {:>15}".format(
        name, train_acc, test_acc, gap, status))

# Visualize train vs test accuracy
fig, ax = plt.subplots(figsize=(12, 6))

model_names = list(results.keys())
x = np.arange(len(model_names))
width = 0.35

train_accs = [results[m]['train_accuracy'] for m in model_names]
test_accs = [results[m]['test_accuracy'] for m in model_names]

bars1 = ax.bar(x - width/2, train_accs, width, label='Train Accuracy', color='steelblue')
bars2 = ax.bar(x + width/2, test_accs, width, label='Test Accuracy', color='coral')

ax.set_xlabel('Model', fontsize=12)
ax.set_ylabel('Accuracy', fontsize=12)
ax.set_title('Train vs Test Accuracy (Overfitting Analysis)', fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(model_names, rotation=45, ha='right')
ax.legend()
ax.set_ylim(0.7, 1.05)

plt.tight_layout()
plt.savefig('09_overfitting_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

---
## 7. Step 6: Feature Importance Analysis

Analyze which features (genes/PCA components) are most influential in the classification:
- Logistic Regression: Model coefficients
- Tree-based models: Feature importance scores
- Map back to original genes

In [ ]:
# =============================================================================
# LOGISTIC REGRESSION COEFFICIENTS
# =============================================================================

print("=" * 70)
print("FEATURE IMPORTANCE ANALYSIS")
print("=" * 70)

print("\n--- Logistic Regression Coefficients ---\n")

lr_model = results['Logistic Regression']['model']
coefficients = lr_model.coef_

print(f"Coefficient matrix shape: {coefficients.shape}")
print(f"(Classes x PCA Components)")

# Calculate average absolute coefficient per component
avg_coef = np.mean(np.abs(coefficients), axis=0)

# Get top 20 most important PCA components
top_20_idx = np.argsort(avg_coef)[-20:][::-1]

print("\nTop 20 Most Important PCA Components (Logistic Regression):")
print("-" * 50)
for i, idx in enumerate(top_20_idx[:10], 1):
    print(f"  {i:2d}. PC{idx+1}: {avg_coef[idx]:.4f}")

In [ ]:
# =============================================================================
# TREE-BASED FEATURE IMPORTANCE
# =============================================================================

print("\n--- Tree-Based Models Feature Importance ---\n")

# Get feature importance from different models
tree_models = ['Decision Tree', 'Random Forest', 'XGBoost', 'Gradient Boosting']

fig, axes = plt.subplots(2, 2, figsize=(16, 12))
axes = axes.flatten()

for idx, model_name in enumerate(tree_models):
    model = results[model_name]['model']
    
    # Get feature importance
    if hasattr(model, 'feature_importances_'):
        importance = model.feature_importances_
    else:
        continue
    
    # Get top 20
    top_idx = np.argsort(importance)[-20:][::-1]
    top_importance = importance[top_idx]
    
    # Plot
    y_pos = np.arange(20)
    axes[idx].barh(y_pos, top_importance[::-1], color='steelblue', edgecolor='black')
    axes[idx].set_yticks(y_pos)
    axes[idx].set_yticklabels([f'PC{i+1}' for i in top_idx[::-1]])
    axes[idx].set_xlabel('Importance', fontsize=11)
    axes[idx].set_title(f'{model_name} - Top 20 Important Components', fontsize=12, fontweight='bold')
    axes[idx].invert_yaxis()

plt.tight_layout()
plt.savefig('10_pca_feature_importance.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# =============================================================================
# MAP BACK TO ORIGINAL GENES
# =============================================================================

print("\n--- Mapping Feature Importance to Original Genes ---\n")

# Use Random Forest importance for mapping
rf_importance = results['Random Forest']['model'].feature_importances_

# Calculate gene importance by weighting PCA loadings
gene_importance = np.zeros(data.shape[1])

for i in range(min(n_components, len(rf_importance))):
    gene_importance += rf_importance[i] * np.abs(pca.components_[i])

# Get top 30 genes
top_30_gene_idx = np.argsort(gene_importance)[-30:][::-1]
top_30_gene_names = data.columns[top_30_gene_idx]
top_30_gene_importance = gene_importance[top_30_gene_idx]

print("Top 30 Most Important Genes for Cancer Classification:")
print("=" * 50)
for i, (gene, imp) in enumerate(zip(top_30_gene_names, top_30_gene_importance), 1):
    print(f"  {i:2d}. {gene}: {imp:.6f}")

In [ ]:
# =============================================================================
# VISUALIZE TOP GENES
# =============================================================================

fig, axes = plt.subplots(1, 2, figsize=(16, 8))

# Bar chart of top 20 genes
top_20_genes = top_30_gene_names[:20]
top_20_importance = top_30_gene_importance[:20]

y_pos = np.arange(20)
axes[0].barh(y_pos, top_20_importance[::-1], color='darkgreen', edgecolor='black')
axes[0].set_yticks(y_pos)
axes[0].set_yticklabels(top_20_genes[::-1])
axes[0].set_xlabel('Importance Score', fontsize=12)
axes[0].set_title('Top 20 Most Important Genes', fontsize=14, fontweight='bold')
axes[0].invert_yaxis()

# Heatmap of top 10 genes across cancer types
top_10_genes = top_30_gene_names[:10]
mean_expression = pd.DataFrame(index=top_10_genes)

for cancer in class_names:
    mask = labels['Class'] == cancer
    mean_expression[cancer] = data.loc[mask, top_10_genes].mean()

# Z-score normalization
mean_expression_norm = (mean_expression.T - mean_expression.T.mean()) / mean_expression.T.std()

sns.heatmap(mean_expression_norm.T, cmap='RdBu_r', center=0, annot=True, fmt='.2f',
            ax=axes[1], cbar_kws={'label': 'Z-Score'})
axes[1].set_title('Top 10 Genes Expression Across Cancer Types', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Cancer Type', fontsize=12)
axes[1].set_ylabel('Gene', fontsize=12)

plt.tight_layout()
plt.savefig('11_gene_importance.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# =============================================================================
# LOGISTIC REGRESSION COEFFICIENTS PER CLASS
# =============================================================================

print("\n--- Logistic Regression: Class-Specific Important Features ---\n")

# Get top genes for each cancer type
for i, cancer_type in enumerate(class_names):
    class_coef = coefficients[i]
    
    # Map to genes
    gene_coef = np.zeros(data.shape[1])
    for j in range(len(class_coef)):
        gene_coef += class_coef[j] * pca.components_[j]
    
    # Get top positive and negative genes
    top_positive_idx = np.argsort(gene_coef)[-5:][::-1]
    top_negative_idx = np.argsort(gene_coef)[:5]
    
    print(f"\n{cancer_type}:")
    print(f"  Positive indicators (higher expression -> more likely {cancer_type}):")
    for idx in top_positive_idx:
        print(f"    - {data.columns[idx]}: {gene_coef[idx]:.4f}")
    print(f"  Negative indicators (lower expression -> more likely {cancer_type}):")
    for idx in top_negative_idx:
        print(f"    - {data.columns[idx]}: {gene_coef[idx]:.4f}")

---
## 8. Step 7: Executive Summary

### Project Goal
Develop a machine learning model to classify cancer types based on RNA-Seq gene expression patterns from the TCGA PANCAN dataset.

### Dataset Overview
- **Source:** The Cancer Genome Atlas (TCGA) PANCAN HiSeq dataset
- **Samples:** 801 patient samples
- **Features:** 20,531 genes (RNA-Seq expression values)
- **Classes:** 5 cancer types (BRCA, COAD, KIRC, LUAD, PRAD)
- **Class Distribution:** Imbalanced (BRCA: 300, COAD: 78)

### Preprocessing
1. No missing values detected
2. StandardScaler normalization applied
3. PCA dimensionality reduction: 20,531 → 530 features (95% variance retained)
4. Stratified train-test split (80-20)

### Model Performance Summary

In [ ]:
# =============================================================================
# EXECUTIVE SUMMARY
# =============================================================================

print("=" * 70)
print("EXECUTIVE SUMMARY")
print("=" * 70)

print("""
PROJECT: Cancer Type Classification from Gene Expression Data
======================================================================

1. OBJECTIVE
   Develop a machine learning pipeline to accurately classify cancer
   types based on RNA-Seq gene expression patterns.

2. DATASET
   - Source: TCGA PANCAN HiSeq dataset
   - Samples: 801 patients
   - Features: 20,531 genes
   - Classes: 5 cancer types (BRCA, COAD, KIRC, LUAD, PRAD)
""")

# Best model summary
best_model_name = eval_df['F1 (Weighted)'].idxmax()
best_metrics = eval_df.loc[best_model_name]

print(f"""3. BEST PERFORMING MODEL: {best_model_name}
   
   Key Metrics:
   - Accuracy:         {best_metrics['Accuracy']:.4f} ({best_metrics['Accuracy']*100:.2f}%)
   - Precision:        {best_metrics['Precision (Weighted)']:.4f}
   - Recall:           {best_metrics['Recall (Weighted)']:.4f}
   - F1-Score:         {best_metrics['F1 (Weighted)']:.4f}
   - AUC-ROC:          {best_metrics['AUC-ROC (OvR)']:.4f}
   - True Positive Rate: {best_metrics['TPR (Avg)']:.4f}
   - True Negative Rate: {best_metrics['TNR (Avg)']:.4f}
""")

print("""4. KEY FINDINGS
   
   a) Model Comparison:
      - Logistic Regression achieved highest accuracy
      - Decision Tree showed signs of overfitting
      - Ensemble methods (Random Forest, XGBoost, Gradient Boosting) performed well
   
   b) Important Genes:""")

print(f"      Top 5 genes: {', '.join(top_30_gene_names[:5])}")

print("""
   c) Cancer Type Separability:
      - KIRC (Kidney) and BRCA (Breast) show distinct expression patterns
      - LUAD (Lung) and COAD (Colon) show some overlap
      - PCA visualization shows good class separation

5. LIMITATIONS
   
   - Class imbalance (BRCA: 300 vs COAD: 78 samples)
   - Gene names are anonymized (gene_0, gene_1, etc.)
   - Limited to 5 cancer types
   - No external validation dataset

6. RECOMMENDATIONS
   
   - Use Logistic Regression for production deployment (best accuracy)
   - Consider ensemble methods for improved robustness
   - Validate on independent datasets before clinical use
   - Investigate identified genes for biological significance
   - Address class imbalance with SMOTE or class weighting

======================================================================
""")

# Final model comparison table
print("\nMODEL COMPARISON TABLE:")
print("=" * 70)
summary_cols = ['Accuracy', 'Precision (Weighted)', 'Recall (Weighted)', 'F1 (Weighted)', 'AUC-ROC (OvR)']
display(eval_df[summary_cols].sort_values('F1 (Weighted)', ascending=False).round(4))

In [ ]:
# =============================================================================
# SAVE FINAL SUMMARY VISUALIZATION
# =============================================================================

fig = plt.figure(figsize=(16, 12))

# Create grid
gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)

# 1. Class distribution
ax1 = fig.add_subplot(gs[0, 0])
colors = sns.color_palette('husl', len(class_distribution))
ax1.bar(class_distribution.index, class_distribution.values, color=colors)
ax1.set_title('Cancer Type Distribution', fontsize=12, fontweight='bold')
ax1.set_xlabel('Cancer Type')
ax1.set_ylabel('Count')

# 2. PCA 2D
ax2 = fig.add_subplot(gs[0, 1])
for i, (cancer_type, color) in enumerate(zip(class_names, colors)):
    mask = y_encoded == i
    ax2.scatter(X_pca[mask, 0], X_pca[mask, 1], c=[color], label=cancer_type, alpha=0.6, s=20)
ax2.set_title('PCA Visualization', fontsize=12, fontweight='bold')
ax2.set_xlabel('PC1')
ax2.set_ylabel('PC2')
ax2.legend(fontsize=8)

# 3. Model accuracy comparison
ax3 = fig.add_subplot(gs[0, 2])
model_accs = eval_df['Accuracy'].sort_values(ascending=True)
ax3.barh(range(len(model_accs)), model_accs.values, color='steelblue')
ax3.set_yticks(range(len(model_accs)))
ax3.set_yticklabels(model_accs.index, fontsize=8)
ax3.set_title('Model Accuracy Comparison', fontsize=12, fontweight='bold')
ax3.set_xlabel('Accuracy')
ax3.set_xlim(0.8, 1.0)

# 4. Confusion matrix (best model)
ax4 = fig.add_subplot(gs[1, 0])
cm = confusion_matrix(y_test, results[best_model_name]['y_test_pred'])
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax4,
            xticklabels=class_names, yticklabels=class_names)
ax4.set_title(f'Confusion Matrix ({best_model_name})', fontsize=12, fontweight='bold')
ax4.set_xlabel('Predicted')
ax4.set_ylabel('Actual')

# 5. Top 10 important genes
ax5 = fig.add_subplot(gs[1, 1])
ax5.barh(range(10), top_30_gene_importance[:10][::-1], color='darkgreen')
ax5.set_yticks(range(10))
ax5.set_yticklabels(top_30_gene_names[:10][::-1], fontsize=8)
ax5.set_title('Top 10 Important Genes', fontsize=12, fontweight='bold')
ax5.set_xlabel('Importance')

# 6. ROC curve for best model
ax6 = fig.add_subplot(gs[1, 2])
y_proba = results[best_model_name]['y_test_proba']
if y_proba is not None:
    for i, (class_name, color) in enumerate(zip(class_names, colors)):
        fpr, tpr, _ = roc_curve(y_test_bin[:, i], y_proba[:, i])
        roc_auc = auc(fpr, tpr)
        ax6.plot(fpr, tpr, color=color, lw=1.5, label=f'{class_name} ({roc_auc:.2f})')
    ax6.plot([0, 1], [0, 1], 'k--', lw=1)
    ax6.set_title(f'ROC Curves ({best_model_name})', fontsize=12, fontweight='bold')
    ax6.set_xlabel('FPR')
    ax6.set_ylabel('TPR')
    ax6.legend(fontsize=7, loc='lower right')

# 7. Summary text
ax7 = fig.add_subplot(gs[2, :])
ax7.axis('off')
summary_text = f"""
EXECUTIVE SUMMARY
{'='*80}
Dataset: TCGA PANCAN RNA-Seq (801 samples, 20,531 genes, 5 cancer types)
Best Model: {best_model_name}
Test Accuracy: {best_metrics['Accuracy']*100:.2f}%  |  F1-Score: {best_metrics['F1 (Weighted)']:.4f}  |  AUC-ROC: {best_metrics['AUC-ROC (OvR)']:.4f}
Top Genes: {', '.join(top_30_gene_names[:5])}
{'='*80}
"""
ax7.text(0.5, 0.5, summary_text, transform=ax7.transAxes, fontsize=11,
         verticalalignment='center', horizontalalignment='center',
         family='monospace', bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.3))

plt.suptitle('Cancer Type Classification - Summary Dashboard', fontsize=16, fontweight='bold', y=1.02)
plt.savefig('12_executive_summary.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n" + "="*70)
print("All visualizations saved successfully!")
print("="*70)

In [ ]:
# =============================================================================
# LIST ALL GENERATED FILES
# =============================================================================

print("\nGenerated Visualization Files:")
print("=" * 40)
files = [
    '01_class_distribution.png',
    '02_eda_visualizations.png',
    '03_pca_variance.png',
    '04_pca_visualization.png',
    '05_pca_3d.png',
    '06_model_comparison.png',
    '07_confusion_matrices.png',
    '08_roc_curves.png',
    '09_overfitting_analysis.png',
    '10_pca_feature_importance.png',
    '11_gene_importance.png',
    '12_executive_summary.png'
]
for f in files:
    print(f"  - {f}")

print("\n" + "="*70)
print("CANCER TYPE CLASSIFICATION PIPELINE COMPLETE!")
print("="*70)